In [1]:
!hostname

cf53b32e0db4


In [2]:
!jupyter --version

4.4.0


https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

potential additional work:
* add strace capture
 * https://github.com/blapid/strace2py
 * https://python-ptrace.readthedocs.io/en/latest/syscall.html
 * https://github.com/dirtyharrycallahan/pystrace

In [3]:
!pip install psutil

In [4]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
print('pandas',pandas.__version__)
import matplotlib.pyplot as plt

pandas 0.23.4


# one-time system details

In [5]:
print(psutil.cpu_freq())

None


In [6]:
pid = os.getpid()
print(pid)

260


In [7]:
py = psutil.Process(pid)
print(py)

psutil.Process(pid=260, name='python', started='01:42:16')


In [8]:
py.name()

'python'

In [9]:
py.exe()

'/opt/conda/bin/python3.6'

In [10]:
py.cwd()

'/home/jovyan/jupyter_notebooks/week_09_timeseries/host_streaming_stats'

In [11]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json']

# system performance

In [12]:
# gives a single float value
psutil.cpu_percent(percpu=True)

[35.8, 88.7]

In [13]:
psutil.cpu_times_percent(percpu=True)

[scputimes(user=14.9, nice=0.0, system=25.4, idle=59.6, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0),
 scputimes(user=68.3, nice=0.0, system=21.1, idle=9.8, iowait=0.8, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0)]

In [14]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

{'user': 1.0, 'nice': 0.0, 'system': 3.0, 'idle': 1.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}
{'user': 3.0, 'nice': 0.0, 'system': 2.0, 'idle': 0.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}


In [15]:
# gives an object with many fields
psutil.virtual_memory()

svmem(total=2096164864, available=1217142784, percent=41.9, used=712642560, free=79200256, active=793460736, inactive=1110032384, buffers=67842048, cached=1236480000, shared=544768, slab=83767296)

In [16]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 2096164864,
 'available': 1217142784,
 'percent': 41.9,
 'used': 712642560,
 'free': 79200256,
 'active': 793460736,
 'inactive': 1110032384,
 'buffers': 67842048,
 'cached': 1236480000,
 'shared': 544768,
 'slab': 83767296}

In [17]:
dict(psutil.swap_memory()._asdict())

{'total': 1073737728,
 'used': 765952,
 'free': 1072971776,
 'percent': 0.1,
 'sin': 0,
 'sout': 765952}

In [18]:
dict(psutil.cpu_stats()._asdict())

{'ctx_switches': 2298170,
 'interrupts': 1076202,
 'soft_interrupts': 439624,
 'syscalls': 0}

In [19]:
psutil.disk_io_counters(perdisk=True).keys()

dict_keys(['loop0', 'loop1', 'loop2', 'loop3', 'loop4', 'loop5', 'loop6', 'loop7', 'nbd0', 'nbd1', 'nbd2', 'nbd3', 'nbd4', 'nbd5', 'nbd6', 'nbd7', 'nbd8', 'nbd9', 'nbd10', 'nbd11', 'nbd12', 'nbd13', 'nbd14', 'nbd15', 'sda', 'sda1', 'sr0', 'sr1', 'sr2'])

In [20]:
dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict())

{'read_count': 0,
 'write_count': 0,
 'read_bytes': 0,
 'write_bytes': 0,
 'read_time': 0,
 'write_time': 0,
 'read_merged_count': 0,
 'write_merged_count': 0,
 'busy_time': 0}

In [21]:
psutil.sensors_temperatures()

{}

In [22]:
psutil.sensors_fans()

{}

In [23]:
psutil.sensors_battery()

### put everything into a single row

In [24]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input, got",type(row))
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)[disk_label]._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [25]:
os.getpid()

260

In [26]:
type(os.getpid())

int

In [27]:
py = psutil.Process(os.getpid())
print(py)

psutil.Process(pid=260, name='python', started='01:42:16')


In [28]:
dict(py.memory_info()._asdict())

{'rss': 110833664,
 'vms': 854986752,
 'shared': 31940608,
 'text': 2232320,
 'lib': 0,
 'data': 189538304,
 'dirty': 0}

In [29]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

memory use: 0.10322189331054688 GB


In [30]:
dict(py.memory_full_info()._asdict())

{'rss': 110833664,
 'vms': 854986752,
 'shared': 31940608,
 'text': 2232320,
 'lib': 0,
 'data': 189538304,
 'dirty': 0,
 'uss': 79613952,
 'pss': 87251968,
 'swap': 0}

In [31]:
py.cpu_percent()

0.0

In [32]:
dict(py.cpu_times()._asdict())

{'user': 1.63, 'system': 0.83, 'children_user': 1.68, 'children_system': 1.44}

In [33]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json']

In [34]:
dict(py.num_ctx_switches()._asdict())

{'voluntary': 1149, 'involuntary': 2014}

In [35]:
dict(py.io_counters()._asdict())

{'read_count': 6385,
 'write_count': 1061,
 'read_bytes': 225280,
 'write_bytes': 4096,
 'read_chars': 29446272,
 'write_chars': 898333}

In [36]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input for row, got",type(row))
        return None
    if not isinstance(pid,int):
        raise TypeError("ERROR: expected a int as input for pid, got",type(pid))
        return None
    try:
        mem_dict=dict(psutil.Process(pid).memory_full_info()._asdict())
    except NoSuchProcess:
        return row
    for k,v in mem_dict.items():
        row['process memory: '+k]=v

    try:
        cpu_times_dict=dict(psutil.Process(pid).cpu_times()._asdict())
    except NoSuchProcess:
        return row
    for k,v in cpu_times_dict.items():
        row['process cpu times: '+k]=v

    try:
        context_switches_dict=dict(psutil.Process(pid).num_ctx_switches()._asdict())
    except NoSuchProcess:
        return row        
    for k,v in context_switches_dict.items():
        row['process number context switches: '+k]=v

    try:
        io_counter_dict = dict(psutil.Process(pid).io_counters()._asdict())
    except NoSuchProcess:
        return row
    for k,v in io_counter_dict.items():
        row['process io counters: '+k]=v
    return row

# gather data and create plots

In [37]:
polling_period_in_seconds = 1
monitoring_duration_in_seconds=10

In [38]:
list_of_rows=[]
start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

elapsed time: 10.346050500869751 seconds


In [39]:
print(df.shape)
df.head()

(10, 328)


,CPU 1 times guest,CPU 1 times guest_nice,CPU 1 times idle,CPU 1 times iowait,CPU 1 times irq,CPU 1 times nice,CPU 1 times softirq,CPU 1 times steal,CPU 1 times system,CPU 1 times user,...,system virtual memory buffers,system virtual memory cached,system virtual memory free,system virtual memory inactive,system virtual memory percent,system virtual memory shared,system virtual memory slab,system virtual memory total,system virtual memory used,time in unix seconds
0,0.0,0.0,19.7,0.0,0.0,0.0,0.7,0.0,52.6,27.0,...,67842048,1236488192,77938688,1110024192,42.0,544768,83800064,2096164864,713895936,1.552960e+09
1,0.0,0.0,93.3,0.0,0.0,0.0,0.0,0.0,3.8,2.9,...,67842048,1236533248,77811712,1110024192,42.0,544768,83845120,2096164864,713977856,1.552960e+09
2,0.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,67842048,1236533248,77811712,1110024192,42.0,544768,83845120,2096164864,713977856,1.552960e+09
3,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,67854336,1236533248,77811712,1110028288,42.0,544768,83845120,2096164864,713965568,1.552960e+09
4,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,67854336,1236533248,77811712,1110028288,42.0,544768,83845120,2096164864,713965568,1.552960e+09


In [40]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [41]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        raise TypeError('ERROR: expected input of type dataframe, got',type(df))
        return None
    if not isinstance(col_name,str):
        raise TypeError('ERROR: expected input of type str, got',type(col_name))
        return None
    plt.plot_date(x=df['datetime'],y=df[col_name])
    plt.xticks(rotation=40)
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

works even when the kernel of the other notebook is restarted

detects when new newbook kernels being added

https://github.com/iiSeymour/nbtop/blob/master/nbtop/main.py

### get a list of all active sessions

In [42]:
r = requests.get('http://localhost:8888/api/sessions')
r.status_code

200

In [43]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
 'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
 'name': 'realtime_data_matplotlib.ipynb',
 'type': 'notebook',
 'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
  'name': 'python3',
  'last_activity': '2019-03-18T11:02:46.614172Z',
  'execution_state': 'idle',
  'connections': 1},
 'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb'}}

### from json returned from API, we can extract the notebook kernel IDs

In [44]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

notebook kernel IDs:
b1c1ad07-ecaa-4181-8258-f9a30ab8eafc
e5598466-2d75-4a55-9b0c-cf7d072159ba
047727fb-9390-40aa-ab15-ad3be786761b
154e7a2a-e9e3-44d8-aece-7d2a915156d0
fdd3dea8-d84d-4adf-8b6d-df46b3aae7d7


### get a list of all pids

In [45]:
print('all processes running:')
for p in psutil.process_iter():
    print(p)

all processes running:
psutil.Process(pid=1, name='tini', started='00:48:53')
psutil.Process(pid=6, name='jupyter-lab', started='00:48:54')
psutil.Process(pid=26, name='python', started='00:49:19')
psutil.Process(pid=162, name='python', started='00:59:34')
psutil.Process(pid=223, name='python', started='01:07:20')
psutil.Process(pid=244, name='python', started='01:36:07')
psutil.Process(pid=260, name='python', started='01:42:16')


### get pid for each notebook

In [46]:
def get_all_kernel_pids():
    dict_of_jupyter_kernel_pids = {}
    for p in psutil.process_iter():
        list_of_commands_per_process = psutil.Process(p.pid).cmdline()
#        print(list_of_commands_per_process)
        if ('ipykernel_launcher' in list_of_commands_per_process):
            dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        
    return dict_of_jupyter_kernel_pids

In [47]:
dict_of_jupyter_kernel_pids = get_all_kernel_pids()
dict_of_jupyter_kernel_pids

{26: '/home/jovyan/.local/share/jupyter/runtime/kernel-b1c1ad07-ecaa-4181-8258-f9a30ab8eafc.json',
 162: '/home/jovyan/.local/share/jupyter/runtime/kernel-154e7a2a-e9e3-44d8-aece-7d2a915156d0.json',
 223: '/home/jovyan/.local/share/jupyter/runtime/kernel-e5598466-2d75-4a55-9b0c-cf7d072159ba.json',
 244: '/home/jovyan/.local/share/jupyter/runtime/kernel-fdd3dea8-d84d-4adf-8b6d-df46b3aae7d7.json',
 260: '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json'}

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [48]:
def add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts):
    if not isinstance(dict_of_jupyter_kernel_pids,dict):
        raise TypeError("ERROR: expected a dict as input for dict_of_jupyter_kernel_pids, got",type(dict_of_jupyter_kernel_pids))
        return None
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
        for notebook_indx in range(len(list_of_kernel_dicts)):
            if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#                print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
                list_of_kernel_dicts[notebook_indx]['pid']=pid
    return list_of_kernel_dicts

In [49]:
list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)

In [50]:
list_of_kernel_dicts

[{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
   'name': 'python3',
   'last_activity': '2019-03-18T11:02:46.614172Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
   'name': 'realtime_data_matplotlib.ipynb'},
  'pid': 26},
 {'id': 'c1db6eca-dcfe-4a4f-a083-3c19eae27a82',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': 'analysis_of_static_timeseries.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'e5598466-2d75-4a55-9b0c-cf7d072159ba',
   'name': 'python3',
   'last_activity': '2019-03-18T11:21:01.855849Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {

### which kernels are not this notebook?

In [51]:
def kernel_dicts_other_notebooks(list_of_kernel_dicts):
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    list_of_kernel_dicts_for_other_notebooks=[]
    for kernel_dict in list_of_kernel_dicts:
        if (str(os.getpid())!=str(kernel_dict['pid'])):
            list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)
    return list_of_kernel_dicts_for_other_notebooks

In [52]:
list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)
list_of_kernel_dicts_for_other_notebooks

[{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
   'name': 'python3',
   'last_activity': '2019-03-18T11:02:46.614172Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
   'name': 'realtime_data_matplotlib.ipynb'},
  'pid': 26},
 {'id': 'c1db6eca-dcfe-4a4f-a083-3c19eae27a82',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': 'analysis_of_static_timeseries.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'e5598466-2d75-4a55-9b0c-cf7d072159ba',
   'name': 'python3',
   'last_activity': '2019-03-18T11:21:01.855849Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {

In [53]:
def get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks,
                          url_for_api_sessions='http://localhost:8888/api/sessions'):
    if not isinstance(list_of_kernel_dicts_for_other_notebooks,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts_for_other_notebooks, got",type(list_of_kernel_dicts_for_other_notebooks))
        return None

    list_of_kernel_dicts = requests.get(url_for_api_sessions).json()
    dict_of_jupyter_kernel_pids = get_all_kernel_pids()
    list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)
    current_list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)

    list_of_pids_in_previous=[]
    for old_dict in list_of_kernel_dicts_for_other_notebooks:
        list_of_pids_in_previous.append(old_dict['pid'])
        for curr_dict in current_list_of_kernel_dicts_for_other_notebooks:
            if (curr_dict['pid'] not in list_of_pids_in_previous):
                list_of_kernel_dicts_for_other_notebooks.append(curr_dict)
    return list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [54]:
monitoring_duration_in_seconds=60 # 60 queries takes 63 seconds

In [55]:
nb_dict={}
list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids([])

start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    system_row = row_for_system_performance(row)

    # detect new notebook kernels
    list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks)
    
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        try:
            nb_dict[other_nb['pid']].append(row)
        except KeyError: # new pid for new kernel
            nb_dict[other_nb['pid']] = []
            nb_dict[other_nb['pid']].append(row)
    time.sleep(polling_period_in_seconds)

    print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

elapsed time: 1.06595778465271 seconds
elapsed time: 2.101262331008911 seconds
elapsed time: 3.135986328125 seconds
elapsed time: 4.1695396900177 seconds
elapsed time: 5.208044767379761 seconds
elapsed time: 6.250380039215088 seconds
elapsed time: 7.289450407028198 seconds
elapsed time: 8.327214002609253 seconds
elapsed time: 9.389519929885864 seconds
elapsed time: 10.423573732376099 seconds
elapsed time: 11.459370851516724 seconds
elapsed time: 12.494818925857544 seconds
elapsed time: 13.539909601211548 seconds
elapsed time: 14.57805585861206 seconds
elapsed time: 15.619434833526611 seconds
elapsed time: 16.650078535079956 seconds
elapsed time: 17.686527252197266 seconds
elapsed time: 18.720250844955444 seconds
elapsed time: 19.759300708770752 seconds
elapsed time: 20.79234290122986 seconds
elapsed time: 21.832957983016968 seconds
elapsed time: 22.870126247406006 seconds
elapsed time: 23.906441688537598 seconds
elapsed time: 24.939666748046875 seconds
elapsed time: 26.92522954940796 s

In [56]:
nb_df.keys()

dict_keys([])

In [57]:
for indx in range(len(list_of_kernel_dicts_for_other_notebooks)):
    nb_pid = list_of_kernel_dicts_for_other_notebooks[indx]['pid']
    list_of_kernel_dicts_for_other_notebooks[indx]['df']=nb_df[nb_pid]

In [58]:
with open("single_process_df_other_notebook_pid_"+str(time.time())+".pkl", 'wb') as handle:
    pickle.dump(list_of_kernel_dicts_for_other_notebooks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#   list_of_kernel_dicts_for_other_notebooks = pickle.load(handle)